In [1]:
using JuMP, HiGHS
#Cake types and months
cakes = [:cheesecake, :blackforest]
months = 1:3

using NamedArrays

#Demand for each type of cake
demand = [40 30 20; 20 30 10]
d= NamedArray(demand, (cakes, months), ("cakes", "month"))


# Cost for each type of cake
cost = [3.00 3.40 3.80; 2.50 2.80 3.40]
c = NamedArray(cost, (cakes, months), ("cakes", "month"))


#Costs for each type of cake
holdcost = Dict(zip(cakes,[0.5, 0.4]))

#Max cakes that can be produced each month
maxproduce = 65

65

In [6]:
mod1 = Model(HiGHS.Optimizer)

@variable(mod1, x[cakes, months] >= 0)
@variable(mod1, y[cakes, months] >= 0)

@constraint(mod1, maxcake[j in months], sum(x[i,j] for i in cakes) <= maxproduce)


@constraint(mod1, month[i in  cakes], x[i,1] - d[i,1] == y[i,1])

@constraint(mod1, flow[i in cakes, j in 2:3], y[i, j - 1]+ x[i,j] - d[i,j] == y[i,j])


@objective(mod1, Min, sum(sum(c[i,j] * x[i,j] for j in months) for i in cakes) + sum(sum(holdcost[i] * y[i,j] for j in months) for i in cakes))
print(mod1)

In [8]:
optimize!(mod1)

Presolving model
7 rows, 10 cols, 18 nonzeros
7 rows, 10 cols, 18 nonzeros
Presolve : Reductions: rows 7(-2); columns 10(-2); elements 18(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.7000000000e+02 Pr: 4(90) 0s
          5     4.6450000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  4.6450000000e+02
HiGHS run time      :          0.02


In [9]:
# cakes per month
println(value.(x))

# cakes in inventory
println(value.(y))

# min cost

println(objective_value(mod1))


2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:cheesecake, :blackforest]
    Dimension 2, 1:3
And data, a 2×3 Matrix{Float64}:
 45.0  25.0  20.0
 20.0  40.0   0.0
2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:cheesecake, :blackforest]
    Dimension 2, 1:3
And data, a 2×3 Matrix{Float64}:
 5.0   0.0  0.0
 0.0  10.0  0.0
464.5


1×3 Matrix{Int64}:
 70  75  65

In [ ]:
#warehouses
numWarehouses = 3
#customers
numCustomers = 3

#Cost of shipping 1 unit from warehouse to customer
costs = [15 35 25;

10 50 40;

20 40 30]

#Units each warehouse can provide
supply = [30 30 30]

#Units each customer need

demand = [40 50 40]

#Cst for each unit of unmet demand
penalty = [70 75 65]